In [1]:
#!/usr/bin/env python3
"""
vterm.py - Part A of ODE3 (Python/solve_ivp version)

Tasks:
1. Projectile motion WITHOUT air resistance: check energy conservation vs time.
2. Turn air resistance ON and estimate terminal velocity for default mass/drag.
3. Study terminal velocity vs mass for 0.001 kg < m < 10 kg and make a plot v_t(m).

Run:
    python vterm.py

This will produce:
    - energy_no_drag.png
    - vterm_vs_mass.png   (you can convert to vterm.pdf if the assignment requires a PDF)
"""

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

g = 9.81  # m/s^2


# ---------------------------------------------------------------------------
# 1. Projectile motion in 2D (x,z) without drag
# ---------------------------------------------------------------------------
def eom_no_drag(t, y, params):
    """
    Equations of motion without drag in 2D:
    state y = [x, z, vx, vz]
    """
    x, z, vx, vz = y
    dxdt = vx
    dzdt = vz
    dvxdt = 0.0
    dvzdt = -g
    return [dxdt, dzdt, dvxdt, dvzdt]


def simulate_no_drag(v0=70.0, theta_deg=45.0, t_max=10.0, dt=0.01, m=1.0):
    """
    Simulate projectile motion without drag and return time, state, and energy.
    """
    theta = np.deg2rad(theta_deg)
    vx0 = v0 * np.cos(theta)
    vz0 = v0 * np.sin(theta)

    y0 = [0.0, 0.0, vx0, vz0]  # [x0, z0, vx0, vz0]

    t_eval = np.arange(0.0, t_max + dt, dt)

    sol = solve_ivp(
        fun=lambda t, y: eom_no_drag(t, y, None),
        t_span=(0.0, t_max),
        y0=y0,
        t_eval=t_eval,
        rtol=1e-9,
        atol=1e-9,
    )

    t = sol.t
    x, z, vx, vz = sol.y
    v2 = vx**2 + vz**2
    E = 0.5 * m * v2 + m * g * z  # total mechanical energy

    return t, x, z, vx, vz, E


def check_energy_conservation():
    """
    Run a no-drag simulation and make a plot of normalized energy vs time.
    """
    t, x, z, vx, vz, E = simulate_no_drag(v0=70.0, theta_deg=45.0, t_max=10.0, dt=0.01, m=1.0)
    E0 = E[0]
    rel_error = (E - E0) / E0

    print(f"Max relative energy error (no drag): {np.max(np.abs(rel_error)):.3e}")

    plt.figure(figsize=(8, 5))
    plt.plot(t, rel_error, label='(E(t) - E0) / E0')
    plt.axhline(0.0, color='k', linestyle='--', linewidth=1)
    plt.xlabel("t [s]")
    plt.ylabel("Relative energy error")
    plt.title("Energy Conservation (No Drag, RK4-like via solve_ivp)")
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.savefig("energy_no_drag.png", dpi=150)
    plt.close()


# ---------------------------------------------------------------------------
# 2. Vertical motion with quadratic drag - terminal velocity
# ---------------------------------------------------------------------------
def eom_vertical_drag(t, y, params):
    """
    Vertical motion with drag (downwards) and gravity.

    State:
        y = [z, v]
    Drag model:
        F_drag = -k * |v| * v  (quadratic drag)
    Equation:
        m dv/dt = -m g + F_drag
    """
    z, v = y
    m = params["m"]
    k = params["k"]

    dzdt = v
    drag = -k * abs(v) * v
    dvdt = -g + drag / m
    return [dzdt, dvdt]


def estimate_terminal_velocity(m=1.0, k=0.1, t_max=50.0):
    """
    Simulate a falling object with drag and estimate terminal velocity
    as the velocity at the end of the simulation.
    """
    y0 = [1000.0, 0.0]  # start high above ground, at rest (z0, v0)
    params = {"m": m, "k": k}

    sol = solve_ivp(
        fun=lambda t, y: eom_vertical_drag(t, y, params),
        t_span=(0.0, t_max),
        y0=y0,
        rtol=1e-8,
        atol=1e-8,
        dense_output=False,
    )

    z, v = sol.y
    vt = v[-1]  # approximate terminal velocity (should be negative)
    return vt


# ---------------------------------------------------------------------------
# 3. Study vt vs mass
# ---------------------------------------------------------------------------
def study_vt_vs_mass():
    """
    Compute terminal velocity vs mass for m in [0.001, 10] kg and plot.
    """
    masses = np.logspace(-3, 1, 40)  # 0.001 to 10 kg (log-spaced)
    k_default = 0.1  # same k as example; you can adjust if desired

    vts = []
    print("Computing vt(m) for masses from 0.001 kg to 10 kg...")
    for m in masses:
        vt = estimate_terminal_velocity(m=m, k=k_default, t_max=60.0)
        vts.append(vt)
        print(f"m = {m:.4f} kg -> vt ≈ {vt:.3f} m/s")

    masses = np.array(masses)
    vts = np.array(vts)

    plt.figure(figsize=(8, 5))
    plt.plot(masses, np.abs(vts), 'o-', label='|v_t| (numerical)')
    plt.xscale('log')
    plt.xlabel("Mass m [kg]")
    plt.ylabel("Terminal speed |v_t| [m/s]")
    plt.title("Terminal Speed vs Mass (Quadratic Drag, k=0.1)")
    plt.grid(True, which='both', alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.savefig("vterm_vs_mass.png", dpi=150)
    plt.close()


# ---------------------------------------------------------------------------
# Main
# ---------------------------------------------------------------------------
def main():
    print("=== Part A: No-drag energy conservation ===")
    check_energy_conservation()
    print("Created: energy_no_drag.png")

    print("\n=== Part A: Terminal velocity for default parameters ===")
    m_default = 1.0
    k_default = 0.1
    vt = estimate_terminal_velocity(m=m_default, k=k_default, t_max=60.0)
    print(f"Estimated terminal velocity for m={m_default} kg, k={k_default}: vt ≈ {vt:.3f} m/s")

    print("\n=== Part A: vt vs mass study ===")
    study_vt_vs_mass()
    print("Created: vterm_vs_mass.png")
    print("\nDone. You can convert vterm_vs_mass.png to vterm.pdf if needed.")


if __name__ == "__main__":
    main()


=== Part A: No-drag energy conservation ===
Max relative energy error (no drag): 1.856e-15
Created: energy_no_drag.png

=== Part A: Terminal velocity for default parameters ===
Estimated terminal velocity for m=1.0 kg, k=0.1: vt ≈ -9.905 m/s

=== Part A: vt vs mass study ===
Computing vt(m) for masses from 0.001 kg to 10 kg...
m = 0.0010 kg -> vt ≈ -0.313 m/s
m = 0.0013 kg -> vt ≈ -0.352 m/s
m = 0.0016 kg -> vt ≈ -0.397 m/s
m = 0.0020 kg -> vt ≈ -0.446 m/s
m = 0.0026 kg -> vt ≈ -0.502 m/s
m = 0.0033 kg -> vt ≈ -0.565 m/s
m = 0.0041 kg -> vt ≈ -0.636 m/s
m = 0.0052 kg -> vt ≈ -0.716 m/s
m = 0.0066 kg -> vt ≈ -0.806 m/s
m = 0.0084 kg -> vt ≈ -0.907 m/s
m = 0.0106 kg -> vt ≈ -1.020 m/s
m = 0.0134 kg -> vt ≈ -1.148 m/s
m = 0.0170 kg -> vt ≈ -1.292 m/s
m = 0.0215 kg -> vt ≈ -1.454 m/s
m = 0.0273 kg -> vt ≈ -1.636 m/s
m = 0.0346 kg -> vt ≈ -1.841 m/s
m = 0.0438 kg -> vt ≈ -2.072 m/s
m = 0.0554 kg -> vt ≈ -2.331 m/s
m = 0.0702 kg -> vt ≈ -2.624 m/s
m = 0.0889 kg -> vt ≈ -2.953 m/s
m = 0.1125 

In [2]:
#!/usr/bin/env python3
"""
baseball1.py - ODE3 Baseball Problem 1 (Python/solve_ivp version)

Model:
    2D projectile (x,z) with drag:
        F_drag = - (b v + c v^2) * v_hat

Parameters:
    d = 0.075 m
    m = 0.145 kg
    g = 9.81 m/s^2
    b = 1.6e-4 * d [N s/m^2]
    c = 0.25 * d**2 [N s^2/m^4]
    theta0 = 1 degree above horizontal

Strike zone target:
    x_end = 18.5 m
    z_target ≈ 0.9 m

Task:
    Find the minimum v0 such that the ball crosses x=18.5 m at z near 0.9 m.
"""

import numpy as np
from scipy.integrate import solve_ivp

g = 9.81


def drag_coefficients(d):
    b = 1.6e-4 * d
    c = 0.25 * d**2
    return b, c


def eom_baseball(t, y, params):
    """
    Equations of motion for baseball with drag in 2D (x,z).
    State: y = [x, z, vx, vz]
    """
    x, z, vx, vz = y
    m = params["m"]
    b = params["b"]
    c = params["c"]

    v = np.sqrt(vx**2 + vz**2) + 1e-16  # avoid div by zero
    # Drag magnitude
    Fd_mag = b * v + c * v**2
    # Drag components (opposite velocity direction)
    Fdx = -Fd_mag * vx / v
    Fdz = -Fd_mag * vz / v

    ax = Fdx / m
    az = Fdz / m - g

    dxdt = vx
    dzdt = vz
    dvxdt = ax
    dvzdt = az

    return [dxdt, dzdt, dvxdt, dvzdt]


def event_plate_x(t, y, params):
    """
    Event: ball reaches home plate x = x_end.
    """
    x_end = params["x_end"]
    x = y[0]
    return x - x_end

event_plate_x.terminal = True
event_plate_x.direction = 1  # trigger when increasing through zero


def event_ground(t, y, params):
    """
    Event: ball hits the ground z = 0.
    """
    z = y[1]
    return z

event_ground.terminal = True
event_ground.direction = -1  # crossing downward


def simulate_pitch(v0, params, t_max=2.0):
    """
    Simulate a pitch with initial speed v0.
    Returns:
        result dict with keys:
            - hit_plate (bool)
            - z_at_plate (float or None)
            - t_plate (float or None)
            - final_state_at_plate (array or None)
    """
    theta0_deg = params["theta0_deg"]
    theta0 = np.deg2rad(theta0_deg)

    x0 = params["x0"]
    z0 = params["z0"]
    vx0 = v0 * np.cos(theta0)
    vz0 = v0 * np.sin(theta0)

    y0 = [x0, z0, vx0, vz0]

    sol = solve_ivp(
        fun=lambda t, y: eom_baseball(t, y, params),
        t_span=(0.0, t_max),
        y0=y0,
        events=[
            lambda t, y: event_plate_x(t, y, params),
            lambda t, y: event_ground(t, y, params),
        ],
        max_step=0.01,
        rtol=1e-8,
        atol=1e-8,
    )

    # Check which event fired
    hit_plate = sol.t_events[0].size > 0
    hit_ground = sol.t_events[1].size > 0

    if hit_plate:
        t_plate = sol.t_events[0][0]
        state_plate = sol.y_events[0][0]
        x_p, z_p, vx_p, vz_p = state_plate
        return {
            "hit_plate": True,
            "z_at_plate": float(z_p),
            "t_plate": float(t_plate),
            "state_plate": state_plate,
        }

    elif hit_ground:
        # Hit the ground before plate
        return {
            "hit_plate": False,
            "z_at_plate": None,
            "t_plate": float(sol.t_events[1][0]),
            "state_plate": None,
        }

    else:
        # Neither event triggered (unlikely for sane parameters)
        return {
            "hit_plate": False,
            "z_at_plate": None,
            "t_plate": float(sol.t[-1]),
            "state_plate": None,
        }


def find_minimum_v0(params, v_min=20.0, v_max=60.0, tol=0.01, z_target=0.9, z_tol=0.05):
    """
    Use bisection to find the minimum v0 such that the ball crosses
    x = x_end with z close to z_target.

    - z_target: desired height at plate (0.9 m)
    - z_tol: acceptable tolerance in meters (e.g., +/- 5 cm)
    """
    def is_good(v0):
        res = simulate_pitch(v0, params)
        if not res["hit_plate"]:
            return False, None
        z_p = res["z_at_plate"]
        return abs(z_p - z_target) <= z_tol, z_p

    # First ensure upper bound works
    ok_high, z_high = is_good(v_max)
    if not ok_high:
        print("Warning: v_max is not sufficient to reach target zone. Try larger v_max.")
        return None, None

    # Ensure lower bound does NOT work
    ok_low, z_low = is_good(v_min)
    if ok_low:
        print("Warning: v_min already reaches target zone. Try smaller v_min.")
        # but still proceed with bisection in [v_min, v_max]
        pass

    a, b = v_min, v_max
    best_v0 = b
    best_z = z_high

    while b - a > tol:
        mid = 0.5 * (a + b)
        ok, z_mid = is_good(mid)
        if ok:
            # This v0 works; try smaller
            best_v0 = mid
            best_z = z_mid
            b = mid
        else:
            # This v0 does not work; need higher speed
            a = mid

    return best_v0, best_z


def main():
    # Parameters
    d = 0.075  # m
    m = 0.145  # kg
    b, c = drag_coefficients(d)

    params = {
        "m": m,
        "b": b,
        "c": c,
        "x0": 0.0,
        "z0": 1.4,
        "theta0_deg": 1.0,
        "x_end": 18.5,
    }

    print("Baseball Problem 1 (Python version)")
    print("------------------------------------")
    print(f"Diameter d       = {d} m")
    print(f"Mass m           = {m} kg")
    print(f"b (linear drag)  = {b:.6e} N·s/m^2")
    print(f"c (quad drag)    = {c:.6e} N·s^2/m^4")
    print(f"Release height   = {params['z0']} m")
    print(f"Plate distance   = {params['x_end']} m")
    print(f"Release angle    = {params['theta0_deg']} deg")

    v0_min, z_at_plate = find_minimum_v0(params, v_min=20.0, v_max=60.0, tol=0.01,
                                         z_target=0.9, z_tol=0.05)

    if v0_min is None:
        print("Failed to find a suitable v0 with given bounds.")
        return

    # Get full info at that v0
    res = simulate_pitch(v0_min, params)
    t_plate = res["t_plate"]
    x_p, z_p, vx_p, vz_p = res["state_plate"]

    speed_at_plate = np.sqrt(vx_p**2 + vz_p**2)

    print("\n=== Solution Summary ===")
    print(f"Minimum initial speed v0      ≈ {v0_min:.3f} m/s")
    print(f"Height at plate z(x=18.5 m)   ≈ {z_p:.3f} m")
    print(f"Time of flight to plate       ≈ {t_plate:.3f} s")
    print(f"Velocity at plate vx, vz      ≈ ({vx_p:.3f}, {vz_p:.3f}) m/s")
    print(f"Speed at plate                ≈ {speed_at_plate:.3f} m/s")

    print("\nCopy/Paste this block into your README.md as required:")
    print("-------------------------------------------------------")
    print(f"v0_min = {v0_min:.3f} m/s")
    print(f"z_at_plate = {z_p:.3f} m")
    print(f"t_plate = {t_plate:.3f} s")
    print(f"vx_plate = {vx_p:.3f} m/s")
    print(f"vz_plate = {vz_p:.3f} m/s")
    print(f"speed_plate = {speed_at_plate:.3f} m/s")


if __name__ == "__main__":
    main()


Baseball Problem 1 (Python version)
------------------------------------
Diameter d       = 0.075 m
Mass m           = 0.145 kg
b (linear drag)  = 1.200000e-05 N·s/m^2
c (quad drag)    = 1.406250e-03 N·s^2/m^4
Release height   = 1.4 m
Plate distance   = 18.5 m
Release angle    = 1.0 deg
Failed to find a suitable v0 with given bounds.
